In [23]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
from sklearn.preprocessing import OneHotEncoder

In [24]:
#Load DataFrame
df = pd.read_csv("charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Remove unnecessary columns
df = df.drop(['NAME','CLASSIFICATION', 'APPLICATION_TYPE','ASK_AMT'], axis = 1)

In [26]:
org_counts = df.ORGANIZATION.value_counts()
org_counts

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [27]:
income_counts = df.INCOME_AMT.value_counts()
income_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [28]:
# Determine which values to replace
replace_inc = list(['1M-5M','5M-10M','50M+','10M-50M'])

# Replace in DataFrame
for inc in replace_inc:
    df.INCOME_AMT = df.INCOME_AMT.replace(inc,"1M+")

replace_inc2 = list(['1-9999','10-24999','25000-99999','100000-499999'])
for inc in replace_inc2:
    df.INCOME_AMT = df.INCOME_AMT.replace(inc, "Thousands")

# Check to make sure binning was successful
df.INCOME_AMT.value_counts()

0              24388
Thousands       7849
1M+             1519
10000-24999      543
Name: INCOME_AMT, dtype: int64

In [29]:
uc_counts = df.USE_CASE.value_counts()
uc_counts

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [30]:
#drop 'other' uc case counts
df = df.loc[df.USE_CASE != "Other", :]


In [31]:
# Import our dependencies
import pandas as pd
import sklearn as skl


# Print out the Country value counts
affiliation_counts = df.AFFILIATION.value_counts()
affiliation_counts

Independent         18477
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [32]:
# Determine which values to replace
replace_affl = list(affiliation_counts[affiliation_counts < 1000].index)

# Replace in DataFrame
for affl in replace_affl:
    df.AFFILIATION = df.AFFILIATION.replace(affl,"Other")


# Check to make sure binning was successful
df.AFFILIATION.value_counts()

Independent         18477
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [33]:
# Generate our categorical variable list
attrition_cat = df.dtypes[df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
df[attrition_cat].nunique()

AFFILIATION               3
USE_CASE                  4
ORGANIZATION              4
INCOME_AMT                4
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [34]:
#Encode Special considerations
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_10000-24999,INCOME_AMT_1M+,INCOME_AMT_Thousands,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [35]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(attrition_cat,1)
df.head()

,EIN,STATUS,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_10000-24999,INCOME_AMT_1M+,INCOME_AMT_Thousands,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,10520599,1,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,10547893,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,10556103,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [36]:
df.dtypes

EIN                               int64
STATUS                            int64
IS_SUCCESSFUL                     int64
AFFILIATION_CompanySponsored    float64
AFFILIATION_Independent         float64
AFFILIATION_Other               float64
USE_CASE_CommunityServ          float64
USE_CASE_Heathcare              float64
USE_CASE_Preservation           float64
USE_CASE_ProductDev             float64
ORGANIZATION_Association        float64
ORGANIZATION_Co-operative       float64
ORGANIZATION_Corporation        float64
ORGANIZATION_Trust              float64
INCOME_AMT_0                    float64
INCOME_AMT_10000-24999          float64
INCOME_AMT_1M+                  float64
INCOME_AMT_Thousands            float64
SPECIAL_CONSIDERATIONS_N        float64
SPECIAL_CONSIDERATIONS_Y        float64
dtype: object

In [37]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Split our preprocessed data into our features and target arrays
y = df["IS_SUCCESSFUL"].values
X = df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [38]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [39]:
len(X_train[0])

19

In [41]:
#Define Model
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 60)                1200      
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 31        
Total params: 3,061
Trainable params: 3,061
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25719 samples
Epoch 1/100
25719/25719 [==============================] - 4s 140us/sample - loss: 513882.9830 - accuracy: 0.5038
Epoch 2/100
25719/25719 [==============================] - 3s 112us/sample - loss: 434010.0721 - accuracy: 0.5041
Epoch 3/100
25719/25719 [==============================] - 3s 107us/sample - loss: 354710.0025 - accuracy: 0.4974
Epoch 4/100
25719/25719 [==============================] - 3s 105us/sample - loss: 300137.0450 - accuracy: 0.4948
Epoch 5/100
25719/25719 [==============================] - 4s 141us/sample - loss: 353330.5078 - accuracy: 0.5010
Epoch 6/100
25719/25719 [==============================] - 3s 108us/sample - loss: 326308.6034 - accuracy: 0.4995
Epoch 7/100
25719/25719 [==============================] - 3s 107us/sample - loss: 325915.7925 - accuracy: 0.4988
Epoch 8/100
25719/25719 [==============================] - 3s 111us/sample - loss: 232224.2115 - accuracy: 0.5019
Epoch 9/100
25719/25719 [==============================] - 3s 107

25719/25719 [==============================] - 2s 92us/sample - loss: 0.6908 - accuracy: 0.5342
Epoch 75/100
25719/25719 [==============================] - 2s 93us/sample - loss: 0.6908 - accuracy: 0.5342
Epoch 76/100
25719/25719 [==============================] - 2s 91us/sample - loss: 0.6909 - accuracy: 0.5342
Epoch 77/100
25719/25719 [==============================] - 2s 91us/sample - loss: 0.6909 - accuracy: 0.5342
Epoch 78/100
25719/25719 [==============================] - 2s 94us/sample - loss: 0.6908 - accuracy: 0.5342
Epoch 79/100
25719/25719 [==============================] - 2s 94us/sample - loss: 0.6909 - accuracy: 0.5342
Epoch 80/100
25719/25719 [==============================] - 2s 94us/sample - loss: 0.6908 - accuracy: 0.5342
Epoch 81/100
25719/25719 [==============================] - 2s 94us/sample - loss: 0.6908 - accuracy: 0.5342
Epoch 82/100
25719/25719 [==============================] - 2s 96us/sample - loss: 0.6908 - accuracy: 0.5342
Epoch 83/100
25719/25719 [======

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8574/1 - 1s - loss: 0.6914 - accuracy: 0.5271
Loss: 0.6918604440251959, Accuracy: 0.5270585417747498


In [45]:
#two_hl = .5271
#tanh = .5437